# 2. Nettoyage du DataFrame/DataFrame cleaning

# Introduction

Ce fichier exécute une étape essentielle du pipeline de science des données : le nettoyage du DataFrame. Cette tâche permet de garantir la qualité, la cohérence et l'exploitabilité des données avant d'entamer toute visualisation ou modélisation prédictive.  
This file performs a key step in the data science pipeline: cleaning the DataFrame. This task ensures the quality, consistency, and usability of the data before any visualization or predictive modeling takes place.  

Voici les étapes de nettoyage traitées dans ce script :  
The cleaning steps handled in this script include:  

1. Importation des bibliothèques nécessaires./Import of necessary libraries.  
2. Chargement et renommage des colonnes./Loading and renaming columns.
3. Fusion avec un autre fichier CSV./Merging with another CSV file.
4. Suppression des doublons et des colonnes inutiles./Removing duplicates and unnecessary columns.  
5. Nettoyage des chaînes de caractères./Cleaning string columns.  
6. Gestion des valeurs manquantes./Handling missing values.  
7. Conversion des variables binaires./Converting binary variables.  
8. Encodage ordinal et catégoriel./Ordinal and categorical encoding.  
9. Encodage One-Hot./One-hot encoding.  
10. Filtrage des prix extrêmes./Filtering out extreme prices.  
11. Sauvegarde du DataFrame nettoyé./Saving the cleaned DataFrame.  

## 1. 1. Importation des bibliothèques / Importing librarie  

Cette étape consiste à importer toutes les bibliothèques nécessaires au nettoyage et au traitement des données.  
This step involves importing all the libraries required for data cleaning and processing.  

In [ ]:
import pandas as pd # Importation de la bibliothèque Pandas/Importing the Pandas library
import os # Importation de la bibliothèque OS/Importing the OS library
from sklearn.preprocessing import LabelEncoder # Importation de LabelEncoder de sklearn/Importing the LabelEncoder from sklearn
import logging  # Pour enregistrer les erreurs et informations/For logging errors and information

- __*import pandas as pd :*__ 
    - Importe la bibliothèque Pandas, utilisée pour manipuler des DataFrames.  
      Imports the Pandas library, used to manipulate DataFrames.  
- __*import os :*__  
    - Importe la bibliothèque standard OS pour interagir avec le système de fichiers.  
      Imports the standard OS library to interact with the file system.  
- __*from sklearn.preprocessing import LabelEncoder :*__  
    - Importe un outil de prétraitement pour transformer des labels en valeurs numériques.  
      Imports a preprocessing tool to convert labels into numerical values.  
- __*import logging :*__  
    - Permet de gérer les messages d'information, de debug et d'erreurs.  
      Enables handling of informational, debug, and error messages.  

## 1.2 Classe utilitaire DataManager / Utility class DataManager

Cette classe statique contient une méthode pour fusionner dynamiquement des colonnes depuis un autre fichier CSV.  
This static class contains a method to dynamically merge columns from another CSV file.  

In [ ]:
class DataManager: # Cette classe est utilisée pour gérer les données/This class is used to manage data
    @staticmethod # Méthode statique pour charger les données/Static method to load data
    def merge_columnsFrom(main_df, path_to_csv, id_col, from_id_col, from_columns_to_merge, verbose=0): # Cette méthode fusionne les colonnes d'un autre fichier CSV dans le DataFrame principal/This method merges columns from another CSV file into the main DataFrame
        """Load columns from other dataset."""
        if verbose: # Afficher les colonnes à fusionner/Display columns to merge
            print(f"DataManager::merge_columnsFrom -> Columns to merge : {from_columns_to_merge}") # Afficher les colonnes à fusionner/Display columns to merge
            print(f"DataManager::merge_columnsFrom -> Columns before merge : {main_df.columns.to_list()}") # Afficher les colonnes avant la fusion/Display columns before merge

        main_df[id_col] = pd.to_numeric(main_df[id_col], errors='coerce').astype('Int64') # Convertir la colonne id en numérique/Convert the id column to numeric

        if not os.path.exists(path_to_csv): # Vérifier si le fichier existe/Check if the file exists
            logging.error(f"File not found: {path_to_csv}") # Enregistrer l'erreur si le fichier n'existe pas/Log the error if the file does not exist
            return None # Retourner None si le fichier n'existe pas/Return None if the file does not exist

        from_df = pd.read_csv(path_to_csv) # Charger le fichier CSV/Load the CSV file

        if from_id_col not in from_df.columns: # Vérifier si la colonne id existe dans le fichier source/Check if the id column exists in the source file
            logging.error(f"Column '{from_id_col}' not found in source file.") # Enregistrer l'erreur si la colonne n'existe pas/Log the error if the column does not exist
            return None # Retourner None si la colonne n'existe pas/Return None if the column does not exist

        from_df = from_df[[from_id_col] + from_columns_to_merge] # Sélectionner les colonnes à fusionner/Select the columns to merge
        from_df = from_df.rename(columns={from_id_col: 'from_id'}) # Renommer la colonne id/ Rename the id column
        from_df = from_df.drop_duplicates(subset=['from_id'], keep='first') # Supprimer les doublons/Remove duplicates

        main_df = main_df.merge(from_df, left_on=id_col, right_on='from_id', how='left') # Fusionner les DataFrames/ Merge the DataFrames
        main_df = main_df.drop(columns=['from_id']) # Supprimer la colonne id fusionnée/Remove the merged id column

        if verbose: # Afficher les colonnes après la fusion/Display columns after merge
            print(f"DataManager::merge_columnsFrom -> columns merged successfully: {main_df.columns.to_list()}") # Afficher les colonnes fusionnées/Display merged columns
        return main_df # Retourner le DataFrame fusionné/Return the merged DataFrame

- __*class DataManager: :*__  
    - Définition d’une classe utilitaire pour gérer des opérations sur les données.  
      Defines a utility class to handle data operations.  

- __*@staticmethod :*__  
    - Indique que la méthode ne dépend pas de l'instance de la classe.  
      Indicates that the method doesn’t depend on a class instance.  

- __*merge_columnsFrom(...) :*__  
    - Méthode pour fusionner des colonnes à partir d’un autre fichier CSV.  
      Method to merge columns from another CSV file.  

- __*main_df[id_col] = pd.to_numeric(...) :*__  
    - Conversion de la colonne d’identifiant en entier, en gérant les erreurs.  
      Converts the identifier column to integers, handling errors.  

- __*if not os.path.exists(path_to_csv) :*__  
    - Vérifie si le fichier à fusionner existe.  
      Checks if the file to merge exists.  

- __*from_df = pd.read_csv(path_to_csv) :*__  
    - Charge le fichier CSV contenant les colonnes à fusionner.  
      Loads the CSV file containing the columns to merge.  
      
if from_id_col not in from_df.columns : vérifie que la colonne de correspondance existe.
Verifies that the matching column exists.
from_df = from_df[[from_id_col] + from_columns_to_merge] : sélectionne uniquement les colonnes nécessaires.
Selects only the required columns.
from_df = from_df.rename(...) : renomme la colonne d’ID source pour faciliter la fusion.
Renames the source ID column to ease the merge.
from_df = from_df.drop_duplicates(...) : garde une seule ligne par identifiant pour éviter les duplications.
Keeps only one row per identifier to avoid duplication.
main_df = main_df.merge(...) : fusionne le DataFrame principal avec celui chargé, à gauche.
Merges the main DataFrame with the loaded one using a left join.
main_df = main_df.drop(columns=['from_id']) : supprime la colonne temporaire utilisée pour la jointure.
Deletes the temporary column used for joining.
if verbose: ... : affiche des logs détaillés si verbose est activé.
Prints detailed logs if verbose is enabled.